In [ ]:
# projeto desenvolvido no Google Colab
!pip install fiona shapely pyproj rtree pygeos geopandas dask-geopandas import_ipynb

In [ ]:
import sqlite3
import pandas as pd
import geopandas as gpd
import numpy as np
import csv

import dask_geopandas as dgpd
from joblib import Parallel, delayed
import multiprocessing

from datetime import date, datetime, timedelta
import time

import os
import h5py

from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/lightnight/"

Mounted at /content/drive
/content/drive/MyDrive/lightnight


In [ ]:
def city_geo():
  ''' funcao que trata a base de dados de shapefile das regioes administrativas'''

  geo_1 = gpd.read_file('resources/gadm_410-levels.gpkg', layer='ADM_1')
  geo_2 = gpd.read_file('resources/gadm_410-levels.gpkg', layer='ADM_2')
  
  country_l2 = geo_2['GID_0'].unique()
  
  geo_1 = geo_1[~geo_1['GID_0'].isin(country_l2)]
  geo_1 = geo_1[['GID_1', 'geometry']]
  geo_1.columns = ['gid', 'geometry']
  
  geo_2 = geo_2[['GID_2', 'geometry']]
  geo_2.columns = ['gid', 'geometry']

  geo = pd.concat([geo_1, geo_2]).reset_index(drop=True)

  return geo


In [ ]:
def extract_and_save(h5_path, _df_br, ano, mes, threshold=0):

  '''funcao que extrai os dados de luminosidade, lat e long dos arquivos .h5 do NASA Black Marble 
  e faz o merge com a base de dados das regioes administrativas. '''
    
  h5 = h5py.File(h5_path, 'r')

  lat = np.array(h5['/HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/lat'])
  lon = np.array(h5['/HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/lon'])
  
  AllAngle_Composite_Snow_Free = np.array(h5['/HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/AllAngle_Composite_Snow_Free'])
  AllAngle_Composite_Snow_Covered = np.array(h5['/HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/AllAngle_Composite_Snow_Covered'])

  NearNadir_Composite_Snow_Covered = np.array(h5['/HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/NearNadir_Composite_Snow_Covered'])
  NearNadir_Composite_Snow_Free = np.array(h5['/HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/NearNadir_Composite_Snow_Free'])

  OffNadir_Composite_Snow_Covered = np.array(h5['/HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/OffNadir_Composite_Snow_Covered'])
  OffNadir_Composite_Snow_Free = np.array(h5['/HDFEOS/GRIDS/VIIRS_Grid_DNB_2d/Data Fields/OffNadir_Composite_Snow_Free'])

  list_radiance = [AllAngle_Composite_Snow_Free, AllAngle_Composite_Snow_Covered,
                   NearNadir_Composite_Snow_Covered, NearNadir_Composite_Snow_Free, 
                   OffNadir_Composite_Snow_Covered, OffNadir_Composite_Snow_Free]
  
  rad_name = ['AllAngle_Composite_Snow_Free', 'AllAngle_Composite_Snow_Covered',
                   'NearNadir_Composite_Snow_Covered', 'NearNadir_Composite_Snow_Free', 
                   'OffNadir_Composite_Snow_Covered', 'OffNadir_Composite_Snow_Free']


  d = {'latitude': [], 'longitude': [], 'radiance': [], 'radiance_type': []}
  df = pd.DataFrame(data=d)

  a = 0
  for radiance in list_radiance:
    df_ = pd.DataFrame(radiance, columns=lon, index=lat)
    df_.replace(65535, np.nan, inplace=True)
    df_ = df_.stack().reset_index()
    df_.columns = ['latitude', 'longitude', 'radiance']
    df_ = df_.dropna()
    df_ = df_.loc[df_['radiance'] > threshold].reset_index(drop=True)
    df_['radiance_type'] = rad_name[a]
    df = pd.concat([df, df_])
    a += 1
  
  # SJOIN utilizado para identificar se determinado pixel de luminosidade, com determinada latitutde e longitude esta contido dentro do multipolygon que representa determinada regiao
  df = df.set_index(['latitude', 'longitude', 'radiance_type'])
  df_light = df.unstack(level=-1)
  df_light = df_light.reset_index(col_level=1)
  df_light.columns = df_light.columns.droplevel(0)
  
  gdf_light = gpd.GeoDataFrame(df_light, geometry=gpd.points_from_xy(df_light.longitude, df_light.latitude))
  gdf_light['geometry'] = gdf_light['geometry'].set_crs(epsg=4326)

  df_merge = gpd.sjoin(gdf_light, _df_br, predicate='within')
  df_merge['count_pixel'] = 1
  df_merge = df_merge.drop(['index_right', 'latitude', 'longitude'], axis=1)

  df_merge = df_merge.groupby(['gid']).sum()
  df_merge = df_merge.reset_index()
  df_merge['date'] = f'{ano}/{mes}'

  if not df_merge.empty:
    df_merge.to_csv('raw_dataset_3_world.csv', sep=';', index=False, mode='a')

In [ ]:
df_br = city_geo()

# definir lista de anos  e meses (dia do ano) a serem processados
anos = [2012, 2013, 2014]
meses = []

for ano in anos:
  lista_meses = os.listdir(f'./VNP46A3/{ano}/')

  for mes in meses:
    lista_tiles = os.listdir(f'./VNP46A3/{ano}/{mes}')
        
    for tile in lista_tiles:
      path = f'./VNP46A3/{ano}/{mes}/{tile}'
      extract_and_save(path, df_br, ano, mes)
    
    print(f'sucesso em coletar {ano}/{mes}')

sucesso em coletar 2019/274
